# Project: Apply Lightweight Fine-Tuning to a Foundation Model

TODO: In this cell, describe your choices for each of the following

* **PEFT technique:** 
```
config = peft.LoraConfig(
    task_type=peft.TaskType.SEQ_CLS, 
    target_modules = ["q_lin", "k_lin", "v_lin", "out_lin"]
)
```
* **Model:** "distilbert-base-uncased"<br/>
https://huggingface.co/distilbert/distilbert-base-uncased<br/>
<br/>
* **Evaluation approach:** Accuracy = Correct Predictions / Total Predictions<br/>
<br/>
* **Fine-tuning dataset:** "imdb"<br/>
https://huggingface.co/datasets/stanfordnlp/imdb<br/>
<br/>

For the sake of clarity, the same format and the same items of the **RUBRIC** are used:<br/>
https://learn.udacity.com/paid-courses/cd13303/lessons/36ab56ae-71a2-4fe2-921d-361d8d188524/concepts/36ab56ae-71a2-4fe2-921d-361d8d188524-project-rubric

## Prepare the Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Load a pretrained HF model

Includes the relevant imports and loads a pretrained Hugging Face model that can be used for sequence classification

The library `transformers` is imported. The model `distilbert-base-uncased` is loaded. The parameters of `DistilBertModel` are frozen. Whereas the parameters of `pre_classifier` and `classifier` are not frozen.

In [1]:
import transformers

model_name = "distilbert-base-uncased"

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, 
    num_labels = 2, id2label = id2label, label2id = label2id)

# Freeze all the parameters in DistilBertModel:
for param in model.base_model.parameters():
    param.requires_grad = False
# Notice the parameters in pre_classifier and classifier are not frozen.

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model `distilbert-base-uncased` is shown.

In [2]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Load and preprocess a dataset

Includes the relevant imports and loads a Hugging Face dataset that can be used for sequence classification. Then includes relevant imports and loads a Hugging Face tokenizer that can be used to prepare the dataset.

A subset of the full dataset may be used to reduce computational resources needed.

The library `datasets` is installed.

In [3]:
import sys
!{sys.executable} -m pip install -q "datasets==2.15.0"

The dataset `imdb` of movie reviews for sentiment analysis is loaded.

In [4]:
import datasets

dataset_name = "imdb"

ds = datasets.load_dataset(dataset_name)
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


The 3 datasets are split for training, validation, and testing.

In [5]:
seed = 1234
train_ds = ds['train'].shuffle(seed = seed)
test_ds_0 = ds['test'].shuffle(seed = seed)
valid_ds = test_ds_0.select(range(12500))
test_ds = test_ds_0.select(range(12500, 25000))

trimmed_size = 300 #100 #20 #200 #2000 OK

train_ds = train_ds.select(range(4 * trimmed_size))
valid_ds = valid_ds.select(range(trimmed_size))
test_ds = test_ds.select(range(trimmed_size))

splits = ['train', 'valid', 'test']
ds = {split:dataset for split, dataset in zip(splits, [train_ds, valid_ds, test_ds])}
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 1200
 }),
 'valid': Dataset({
     features: ['text', 'label'],
     num_rows: 300
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 300
 })}

The tokenizer for the model `distilbert-base-uncased` is loaded.

In [6]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

The 3 datasets `train`, `valid`, and `test` are tokenized.

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation = True)


tokenized_ds = {split:ds[split].map(preprocess_function, batched = True) for split in splits}

This function helps to show samples from the datasets.

In [8]:
def print_example(split, index):
    sample = tokenized_ds[split][index]
    for key in sample:
        print(f'{key}: {sample[key]}\n')

The first sample of the dataset `train` is shown.

In [9]:
print_example('train', 0)

text: If you love cult 70's Sci-fi the way I do, or if you like movies such as "Repo Man" or "Buckaroo Bonzai" than you're going to love this one. It's a stream of consciousness 70's Sci-fi spectacular, including a 22nd century junkyard and the Earth a million years from now. This movie is pure 70's. Put on Steve Miller's "Fly Like An Eagle" or Pink Floyd's "Dark Side Of The Moon" and you're ready to go!

label: 1

input_ids: [101, 2065, 2017, 2293, 8754, 3963, 1005, 1055, 16596, 1011, 10882, 1996, 2126, 1045, 2079, 1010, 2030, 2065, 2017, 2066, 5691, 2107, 2004, 1000, 16360, 2080, 2158, 1000, 2030, 1000, 10131, 10464, 2080, 14753, 25290, 1000, 2084, 2017, 1005, 2128, 2183, 2000, 2293, 2023, 2028, 1012, 2009, 1005, 1055, 1037, 5460, 1997, 8298, 3963, 1005, 1055, 16596, 1011, 10882, 12656, 1010, 2164, 1037, 13816, 2301, 18015, 14132, 1998, 1996, 3011, 1037, 2454, 2086, 2013, 2085, 1012, 2023, 3185, 2003, 5760, 3963, 1005, 1055, 1012, 2404, 2006, 3889, 4679, 1005, 1055, 1000, 4875, 2066,

The first sample of the dataset `valid` is shown.

In [10]:
print_example('valid', 0)

text: 1891: Stalwart, morally upright military doctor Lieutenant Claude de Ross (solid Claudio Cassinelli) and several other shipwreck survivors wash ashore on a remote tropical island that's governed with an iron fist by the ruthless and sadistic Edmund Rackham (superbly played to the deliciously slimy hilt by Richard Johnson), who lives on the island with the feisty Amanda Martin (a winningly spunky performance by the ravishing Barbara Bach) and her unhinged rogue biologist father Professor Ernest Martin (a marvelously dotty portrayal by Joseph Cotten). Moreover, de Ross discovers that Professor Martin has control over a dangerous race of fishman beasts who are being exploited as slave labor by Rackham. Director/co-writer Sergio Martino relates the lively and absorbing story at a constant snappy pace, offers a flavorsome evocation of the lush and remote tropical setting, does an expert job of creating and maintaining a creepy and mysterious atmosphere in the spooky opening third, fur

The first sample of the dataset `test` is shown.

In [11]:
print_example('test', 0)

text: Any fan of Russian cinema will have great difficulty in believing the sub-par performances phoned-in by Mashkov and Bodrov Jr., and will perhaps be utterly perplexed by Bodrov Sr.'s hackneyed and confusing script, which is coupled with uncharacteristically weak direction. Most of the characters wander through the movie as though they have no idea who they are or what they are doing. It is also sad to see that Jennifer Jason Leigh's acting skills have not advanced one iota since FAST TIMES AT RIDGEMONT HIGH, and her screen exposure is mercifully limited. This is a terrible mafia movie; so much so that it makes the GODFATHER III look like a winner in the genre. To see the key Russians at their best, check out Bodrov Sr.'s work on PRISONER OF THE MOUNTAINS (which also features Jr.), Mashkov's turn in VOR (THE THIEF), and Bodrov Jr.'s new criminal in BRAT.

label: 0

input_ids: [101, 2151, 5470, 1997, 2845, 5988, 2097, 2031, 2307, 7669, 1999, 8929, 1996, 4942, 1011, 11968, 4616, 3042

### Evaluate the pretrained model

At least one classification metric is calculated using the dataset and pretrained model.

As a first experiment, a Foundation Model was going to do the classification as this lesson suggested:<br/>
**Exercise: Use a Foundation Model to Build a Spam Email Classifier**<br/>
https://learn.udacity.com/paid-courses/cd13303/lessons/9a82bcf5-6259-49b7-96c3-5629eb75c820/concepts/58275834-f755-4edc-a045-83e88c61756f

However, only commercial LLMs like `ChatGPT 4` or `Google Gemini` are intelligent enough to answer such prompts in a correct way. For example, the model `distilbert-base-uncased` is not intelligent enough to generate JSON code with the correct answers.

So, a different approach was used like the approach used in this lesson:<br/>
**Exercise: Create a BERT Sentiment Classifier**<br/>
https://learn.udacity.com/paid-courses/cd13303/lessons/cf85c5fa-33bb-4da4-b64b-f3d996d2ada3/concepts/0cff5a1c-6dd3-4fb6-afe8-dad7ac382dfe

An object `transformers.Trainer` is instantiated in order to evaluate the model. But this object is not trained.

In [12]:
import numpy as np
import transformers


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

EPOCHS = 10

training_args = transformers.TrainingArguments(
    output_dir = "./data/sentiment_analysis",
    learning_rate = 1e-3,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = EPOCHS,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy'
)

trainer = transformers.Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["valid"],
    tokenizer = tokenizer,
    data_collator = transformers.DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics = compute_metrics
)

#### Evaluation of Pretrained Model

The model is evaluated with the datasets `valid` and `test`, producing bad accuracies close to 50%, which is random guessing for 2 variables. These accuracies are bad in comparison to the accuracies produced by the PEFT model. See <a href="LightweightFineTuning2.ipynb#Evaluate-the-fine-tuned-model">Evaluate the fine-tuned model</a>.

In [13]:
trainer.evaluate(tokenized_ds["valid"])

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6984267234802246,
 'eval_accuracy': 0.4866666666666667,
 'eval_runtime': 6.0332,
 'eval_samples_per_second': 49.725,
 'eval_steps_per_second': 3.149}

In [14]:
trainer.evaluate(tokenized_ds["test"])

{'eval_loss': 0.7003757953643799,
 'eval_accuracy': 0.47333333333333333,
 'eval_runtime': 4.9691,
 'eval_samples_per_second': 60.373,
 'eval_steps_per_second': 3.824}

## Perform Lightweight Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

### Create a PEFT model

Includes the relevant imports, initializes a Hugging Face PEFT config, and creates a PEFT model using that config

The library `peft` is imported. And the LORA configuration is specified.

In [15]:
import peft

lora_config = peft.LoraConfig(task_type=peft.TaskType.SEQ_CLS, target_modules = ["q_lin", "k_lin", "v_lin", "out_lin"])

The PEFT model is created based on the previous model `distilbert-base-uncased` and on the LORA configuration.

In [16]:
peft_model = peft.get_peft_model(model, lora_config)

The PEFT model is shown. So, it's possible to see the additional layers and neurons added.

In [17]:
peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=False)
                

### Train the PEFT model

The model is trained for at least one epoch using the PEFT model and dataset

Here, the total number of parameters and the number of trainable parameters of the PEFT model are shown.

In [18]:
peft_model.print_trainable_parameters()

trainable params: 1,479,172 || all params: 67,842,052 || trainable%: 2.180317305260755


The trainer is configured with the PEFT model.

In [19]:
trainer = transformers.Trainer(
    model = peft_model,
    args = training_args,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["valid"],
    tokenizer = tokenizer,
    data_collator = transformers.DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics = compute_metrics
)

The PEFT model is trained.

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.310175,0.866667
2,No log,0.281342,0.880000
3,No log,0.403397,0.870000
4,No log,0.474329,0.863333
5,No log,0.669765,0.870000
6,No log,0.657609,0.876667
7,0.165900,0.757528,0.873333
8,0.165900,0.805514,0.876667
9,0.165900,0.848529,0.880000
10,0.165900,0.871084,0.876667


TrainOutput(global_step=750, training_loss=0.11215173467000325, metrics={'train_runtime': 527.269, 'train_samples_per_second': 22.759, 'train_steps_per_second': 1.422, 'total_flos': 1622308700160000.0, 'train_loss': 0.11215173467000325, 'epoch': 10.0})

### Save the PEFT model

Fine-tuned parameters are saved to a separate directory. The saved weights directory should be in the same home directory as the notebook file.

The best PEFT model is saved in the directory `my-peft-distilbert`.

In [21]:
saved_peft_model_name = "my-peft-distilbert"

peft_model.save_pretrained(saved_peft_model_name)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

### Load the saved PEFT model

Includes the relevant imports then loads the saved PEFT model

The best PEFT model is loaded from the directory `my-peft-distilbert`.

In [22]:
peft_model = peft.AutoPeftModelForSequenceClassification.from_pretrained(saved_peft_model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The loaded PEFT model is shown in order to guarantee that everything works correctly.

In [23]:
peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=False)
                

The loaded PEFT model is copied to the CUDA device

In [24]:
peft_model.to("cuda")
trainer.model = peft_model

### Evaluate the fine-tuned model

Repeats the earlier evaluation process (same metric(s) and dataset) to compare the fine-tuned version to the original version of the model

The loaded PEFT model is evaluated with the datasets `valid` and `test`, producing much better accuracies than the accuracies produced by the previous pretrained model. See <a href="LightweightFineTuning2.ipynb#Evaluation-of-Pretrained-Model">Evaluation of Pretrained Model</a>.

In [25]:
trainer.evaluate(tokenized_ds["valid"])

{'eval_loss': 0.2813419997692108,
 'eval_accuracy': 0.88,
 'eval_runtime': 5.4539,
 'eval_samples_per_second': 55.007,
 'eval_steps_per_second': 3.484,
 'epoch': 10.0}

In [26]:
trainer.evaluate(tokenized_ds["test"])

{'eval_loss': 0.25368162989616394,
 'eval_accuracy': 0.92,
 'eval_runtime': 5.4377,
 'eval_samples_per_second': 55.171,
 'eval_steps_per_second': 3.494,
 'epoch': 10.0}

Here, some samples are shown with their labels and their predictions. Some predictions are correct and other predictions are wrong.

In [27]:
import pandas as pd

df = pd.DataFrame(tokenized_ds["test"])
df = df[["text", "label"]]
df["text"] = df["text"].str.replace("<br />", " ")
predictions = trainer.predict(tokenized_ds["test"])
df["predicted_label"] = np.argmax(predictions[0], axis=1)
pd.set_option("display.max_colwidth", None)

df.head(50)

,text,label,predicted_label
0,"Any fan of Russian cinema will have great difficulty in believing the sub-par performances phoned-in by Mashkov and Bodrov Jr., and will perhaps be utterly perplexed by Bodrov Sr.'s hackneyed and confusing script, which is coupled with uncharacteristically weak direction. Most of the characters wander through the movie as though they have no idea who they are or what they are doing. It is also sad to see that Jennifer Jason Leigh's acting skills have not advanced one iota since FAST TIMES AT RIDGEMONT HIGH, and her screen exposure is mercifully limited. This is a terrible mafia movie; so much so that it makes the GODFATHER III look like a winner in the genre. To see the key Russians at their best, check out Bodrov Sr.'s work on PRISONER OF THE MOUNTAINS (which also features Jr.), Mashkov's turn in VOR (THE THIEF), and Bodrov Jr.'s new criminal in BRAT.",0,0
1,"Let's get one thing straight, this gets an 7 out of 10 not on a normal scale, but out of the bad movie scale. this is the kind of movie you rent on purpose, where you intentionally walk in knowing that it is a horrendous knockoff and shun'd by everyone else. I went in with one promise from the movie, that there will be snakes on a train, and it Delivers! The gore itself is really good, and the characters have awesome roles. Come on, it has everything from stoned train pilots to teenage girls trafficking drugs, even a Electrical Engineer getting his pimp on! You get to see some topless nudity, explosions, snakes, gore, and a Mexican main lead running around curing his girlfriend by hitting his crack pipe and blowing the smoke in her face!! As I mentioned and many others have, the movie pacing is a bit off, but respectable nonetheless. Movies like this keep our group tradition of banding together and all chipping in a buck or two to watch masterpieces such as this. There can be no better time spent then coming together to enjoy a good bad movie. It could learn a thing or two from the likes of other such fine flicks as Alien Lock-down or Boa vs Python, but those are some big shoes to fill. A solid 7 out of 10.",1,1
2,"Hard to imagine what they were thinking of when they made this movie (i.e., the writers, directors, producers, actors, editors, etc.). Christopher Plummer, veteran of 129 movies, frolics along among scores of other actors with apparently no more motivation than to collect a paycheck. I guess there is nothing wrong with that, but once they are paid that doesn't mean anyone has to watch it. It bugs me that there are actually good reviews for this movie here at imdb. Art? If you want to see art go to an art gallery, don't watch this movie. Comedy? Watch a re-run of the Flintstones, about the same plot with less time wasted. Dabney Coleman gives his usual performance, for better or worse. And some of the young actors may have gotten some good experience from doing this movie. But Plummer???? It was embarrassing to watch his performance, in fact I was positively transfixed on him throughout the movie, knowing this was Plummer of Sound of Music fame! I see from his bio that he called Sound of Music ""sound of mucus"", so guess he didn't like it as much as the 100's of millions who liked him in it. I wonder if today he was asked, how do you rate Sound of Music compared to Where the Heart Is, what would he say.....? Probably something like ""Where the Money Is""....",0,0
3,"Most of the silent films I've seen have been serious in nature, so it was fun to see one with a comic touch. The setting and some of the scenes for ""The Beloved Rogue"" were reminiscent of 1923's ""The Hunchback of Notre Dame"" relative to the Paris street scenes and the celebration of the 'King of Fools'. John Barrymore portrays France's greatest poet Francois Villon in a characterization that ranges quite broadly from virtual slapstick to romantically tender; that 'water into wine' bit early in the picture was rather amusing. It seems that times never change, and i

Here, only wrong predictions are selected. And then these samples are shown just to emphasize on the wrong predictions of the model.

In [28]:
df[df["label"] != df["predicted_label"]].head(50)

text  \
11                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      